In [1]:
# # Clone xvr repo and move into the directory
# !git clone https://github.com/eigenvivek/xvr.git
# %cd xvr

# # Install PyTorch for CUDA 11.8 (Colab T4 GPU)
# !pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# # Install xvr and dependencies
# !pip install git+https://github.com/eigenvivek/xvr.git
!pip install wandb nibabel torchio tqdm



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import torchvision
import torchaudio
import nibabel as nib
import torchio
import tqdm
import wandb

# Check CUDA availability and GPU info
print("Torch CUDA available:", torch.cuda.is_available())
print("Torch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA Version:", torch.version.cuda)

# Check xvr import
try:
    import xvr
    print("✅ xvr imported successfully")
except ImportError as e:
    print("❌ xvr import failed:", e)


Torch CUDA available: True
Torch version: 2.5.1+cu121
GPU Name: NVIDIA RTX 6000 Ada Generation
CUDA Version: 12.1
✅ xvr imported successfully


In [3]:
import os

# Set a value
os.environ["WANDB_API_KEY"] = "524773117955f64051f5e2bd2ecef2642a677a34"

In [4]:
# Fix wandb login inside xvr training script (do this only once)
!sed -i 's/wandb.login(key=os.environ\["WANDB_API_KEY"\])/wandb.login()/' /usr/local/lib/python3.*/dist-packages/xvr/commands/train.py

'sed' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import sys
import os

# Add the path to xvr/src so you can import xvr.*
sys.path.append(os.path.join(os.getcwd(), 'xvr', 'src'))


In [6]:
import sys
import os

# Step 1: Add xvr/src to Python path so imports work
sys.path.append(os.path.join(os.getcwd(), 'xvr', 'src'))

# Step 2: Patch wandb.login line in local train.py
train_py = os.path.join(os.getcwd(), 'xvr', 'src', 'xvr', 'commands', 'train.py')

if os.path.exists(train_py):
    with open(train_py, 'r') as f:
        lines = f.readlines()

    with open(train_py, 'w') as f:
        for line in lines:
            f.write(line.replace(
                'wandb.login(key=os.environ["WANDB_API_KEY"])',
                'wandb.login()'
            ))

    print(f"✅ Patched wandb.login() in: {train_py}")
else:
    print("❌ train.py not found. Check the path.")


✅ Patched wandb.login() in: C:\Users\aksha\Robossis\xvr\src\xvr\commands\train.py


In [7]:
# !xvr train \
#   --inpath "SE000002.nii" \
#   --outpath "output" \
#   --r1 -45 45 --r2 -45 45 --r3 -15 15 \
#   --tx -200 200 --ty -450 -1000 --tz -200 200 \
#   --sdd 1000.0 \
#   --height 128 \
#   --delx 0.316 \
#   --batch_size 116 \
#   --n_epochs 1000 \
#   --n_batches_per_epoch 100

In [8]:
import xvr
!xvr --help

Usage: xvr [OPTIONS] COMMAND [ARGS]...

  xvr is a PyTorch package for training, fine-tuning, and performing 2D/3D
  X-ray to CT/MR registration with pose regression models.

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  train     Train a pose regression model from scratch.
  restart   Restart model training from a checkpoint.
  finetune  Optimize a pose regression model for a specific patient.
  register  Use gradient-based optimization to register XRAY to a CT/MR.
  animate   Animate the trajectory of iterative optimization.
  dcm2nii   Convert a DICOMDIR to a NIfTI file.


In [ ]:
import subprocess

# Updated Windows-style absolute paths
input_path = "SE000002.nii"
output_path = "output"

command = [
    "xvr", "train",
    "--inpath", input_path,
    "--outpath", output_path,
    "--r1", "-45", "45",
    "--r2", "-45", "45",
    "--r3", "-15", "15",
    "--tx", "-200", "200",
    "--ty", "-450", "-1000",
    "--tz", "-200", "200",
    "--sdd", "1000.0000000000000",
    "--height", "128",
    "--delx", "2.42",
    "--batch_size", "116",
    "--n_epochs", "1000",
    "--n_batches_per_epoch", "100",
    "--project", "RXVR1.1"
]

# Run the command and print output live
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in process.stdout:
    print(line, end='')  # print output as it streams


In [ ]:
process.kill()
print("✅ Training process terminated.")

import torch
import gc

# Garbage collect unreferenced objects
gc.collect()

# Clear unused memory from PyTorch's CUDA allocator
torch.cuda.empty_cache()

print("✅ GPU cache cleared.")


In [ ]:
!xvr train --help

In [ ]:
!xvr animate \
  --inpath "/content/drive/MyDrive/SE000002.nii" \
  --checkpoint "/content/drive/MyDrive/Trained Registration model/xvr_0125.pth" \
  --outpath "/content/drive/MyDrive/Trained Registration model" \
  --height 128 --delx 0.5